In [7]:
#create SVC model to classification spots images of spots parking
import os
import pickle

from skimage.io import imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# prepare data
input_dir = r'E:\Download\New folder\clf-data\clf-data'
categories = ['empty', 'not_empty']

data = []
labels = []
for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path)
        img = resize(img, (15, 15))
        data.append(img.flatten())
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)

# train / test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

# train classifier
classifier = SVC()

parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

pickle.dump(best_estimator, open('./model.p', 'wb'))

100.0% of samples were correctly classified


In [12]:
import pickle

from skimage.transform import resize
import numpy as np
import cv2


EMPTY = True
NOT_EMPTY = False

MODEL = pickle.load(open("model.p", "rb"))


def empty_or_not(spot_bgr): #prediction with SVC model

    flat_data = []

    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY


def get_parking_spots_bboxes(connected_components): #storing coordinates in x1,y1,w,h
    (totalLabels, label_ids, values, centroid) = connected_components
    #print(values)

    slots = []
    coef = 1
    for i in range(1, totalLabels):

        # Now extract the coordinate points
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        slots.append([x1, y1, w, h])

    return slots


In [13]:
import cv2
import matplotlib.pyplot as plt
import numpy as np



def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))


mask = 'mask_1920_1080.png'
video_path = 'parking_1920_1080.mp4'


mask = cv2.imread(mask, 0)

cap = cv2.VideoCapture(video_path)

connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S) # return coordinate of connected pixels

spots = get_parking_spots_bboxes(connected_components)

spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30
while ret:
    ret, frame = cap.read()

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            #print(len(spots))
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])
        #print(len(diffs))
        print([diffs[j] for j in np.argsort(diffs)][::-1]) # j will be the index for each spot 396 and we will take the avarage value for each index from diff list by j index
       # plt.hist([diffs[j] / np.amax(diffs) for j in np.argsort(diffs)][::-1],bins = 20)
       # plt.show()
    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
            #print(len(arr_))
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            spot_status = empty_or_not(spot_crop)

            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1

cap.release()
cv2.destroyAllWindows()

[7.591036414565821, 7.32528180354268, 7.1615858737298055, 7.014159586873248, 6.439095928226351, 6.401851851851859, 6.110378634212296, 6.088122605363992, 6.0571895424836555, 5.906423331544119, 5.861111111111114, 5.743794769282033, 5.742295518907213, 5.624070317782298, 5.597826086956523, 5.5755772005772, 5.574183006535947, 5.572663139329805, 5.556302521008391, 5.540896218557393, 5.507366035292208, 5.419623521572547, 5.369315342328846, 5.301759834368532, 5.296618357487915, 5.263235294117649, 5.246798029556643, 5.236392914653791, 5.215059137098109, 5.198951994590942, 5.186162870945481, 5.165307971014485, 5.131535947712422, 5.116801893171058, 5.0926015085861, 5.053117380703583, 5.027260179434094, 5.019162640901769, 4.948275862068954, 4.9243156199678, 4.918242296918777, 4.901328273244786, 4.896881287726359, 4.754267310789061, 4.750700280112056, 4.7482492997198875, 4.736815415821496, 4.654303599374032, 4.6359477124183, 4.6250862663906105, 4.598200899550221, 4.5933123249299825, 4.5865905431122

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
